In [48]:
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input
from glob import glob

In [46]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [73]:
from keras.models import load_model
network = load_model('/content/drive/MyDrive/ChestVGG_SGD.h5')

In [74]:
network.layers[40].output

<KerasTensor: shape=(None, 7, 7, 1024) dtype=float32 (created by layer 'concatenate_11')>

In [75]:
from tensorflow.keras.layers import Input
model = Model(inputs=network.input, outputs=network.layers[40].output)

In [76]:
train_path = '/content/drive/MyDrive/Dataset/train'
test_path = '/content/drive/MyDrive/Dataset/test'

In [77]:
folders = glob('/content/drive/MyDrive/Dataset/train/*')
print(len(folders))

2


In [78]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

In [79]:
test_datagen = ImageDataGenerator(
    rescale=1./255)

In [80]:
train_set = train_datagen.flow_from_directory(train_path,
                                                target_size = (224, 224),
                                                batch_size=1,
                                                shuffle=False,
                                                 class_mode = 'categorical')

Found 4686 images belonging to 2 classes.


In [81]:
test_set = test_datagen.flow_from_directory(test_path,
                                            target_size = (224, 224),
                                            shuffle=False,
                                            batch_size=1,
                                            class_mode = 'categorical')

Found 1170 images belonging to 2 classes.


In [82]:
features=model.predict(train_set, steps=4686, verbose=1)

4686/4686 [==============================] - 415s 89ms/step


In [83]:
features_flattened=[]
for i in features:
  a=i.ravel()
  features_flattened.append(a)

In [84]:
from sklearn.svm import LinearSVC
svc=LinearSVC(C=1, dual=False, max_iter=6000)
model_svc=svc.fit(features_flattened, train_set.classes)

In [85]:
features_extracted_test=model.predict(test_set, steps=1170, verbose=1)

1170/1170 [==============================] - 104s 89ms/step


In [86]:
features_extracted_test_flattened=[]
for i in features_extracted_test:
  a=i.ravel()
  features_extracted_test_flattened.append(a)

In [87]:
preds=model_svc.predict(features_extracted_test_flattened)

In [88]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
y_test=test_set.classes
cf_matrix=confusion_matrix(y_test, preds)
print(cf_matrix)
from sklearn.metrics import accuracy_score
accuracy_score(y_test, preds)

[[308   8]
 [ 31 823]]


0.9666666666666667

In [89]:
from sklearn.ensemble import RandomForestClassifier
random_forest=RandomForestClassifier(random_state=0)
model_RF=random_forest.fit(features_flattened, train_set.classes)


In [90]:
preds_RF=model_RF.predict(features_extracted_test_flattened)

In [91]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
y_test=test_set.classes
cf_matrix=confusion_matrix(y_test, preds_RF)
print(cf_matrix)
from sklearn.metrics import accuracy_score
accuracy_score(y_test, preds_RF)

[[304  12]
 [ 22 832]]


0.9709401709401709

In [92]:
from sklearn.linear_model import LogisticRegression
logit=LogisticRegression(random_state=0, max_iter=6000)
model_logit=logit.fit(features_flattened, train_set.classes)


In [93]:
preds_logit=model_logit.predict(features_extracted_test_flattened)

In [94]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
y_test=test_set.classes
cf_matrix=confusion_matrix(y_test, preds_logit)
print(cf_matrix)
from sklearn.metrics import accuracy_score
accuracy_score(y_test, preds_logit)

[[310   6]
 [ 31 823]]


0.9683760683760684

In [122]:
from sklearn.neighbors import KNeighborsClassifier
knn_optimized=KNeighborsClassifier(n_neighbors=19).fit(features_flattened, train_set.classes)

In [123]:
preds_knn=knn_optimized.predict(features_extracted_test_flattened)

In [124]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
y_test=test_set.classes
cf_matrix=confusion_matrix(y_test, preds_knn)
print(cf_matrix)
from sklearn.metrics import accuracy_score
accuracy_score(y_test, preds_knn)

[[311   5]
 [ 29 825]]


0.9709401709401709

In [101]:
from sklearn.neighbors import KNeighborsClassifier
knn_optimized=KNeighborsClassifier(n_neighbors=5).fit(features_flattened, train_set.classes)

In [102]:
preds_knn=knn_optimized.predict(features_extracted_test_flattened)

In [103]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
y_test=test_set.classes
cf_matrix=confusion_matrix(y_test, preds_knn)
print(cf_matrix)
from sklearn.metrics import accuracy_score
accuracy_score(y_test, preds_knn)

[[311   5]
 [ 26 828]]


0.9735042735042735

In [98]:
from sklearn.naive_bayes import GaussianNB
classifier_NB=GaussianNB()
model_NB=classifier_NB.fit(features_flattened, train_set.classes)

In [99]:
preds_NB=model_NB.predict(features_extracted_test_flattened)

In [100]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
y_test=test_set.classes
cf_matrix=confusion_matrix(y_test, preds_NB)
print(cf_matrix)
from sklearn.metrics import accuracy_score
accuracy_score(y_test, preds_NB)

[[234  82]
 [ 30 824]]


0.9042735042735043